## Day 23: Unstable Diffusion

[![nbviewer](https://raw.githubusercontent.com/jupyter/design/master/logos/Badges/nbviewer_badge.svg)](https://nbviewer.org/github/mazharenko/AoC-2022/tree/HEAD/notebooks/day23/puzzle.ipynb)

### Preparation

In [133]:
#!value --name sampleRaw
....#..
..###.#
#...#.#
.#...##
#.###..
##.#.##
.#..#..

In [134]:
open System.Collections.Generic
#load "../common/common.fsx"

type Elves = { Coordinates: HashSet<Point> }

let parse s = 
    let points = 
        Pattern1.read (Seq.toArray) s
        |> array2D
        |> Array2D.indexed
        |> Array2D.toSeq
        |> Seq.filter (snd >> ((=)'#'))
        |> Seq.map (fst >> Point)
    { Coordinates = HashSet(points) }
    

In [135]:
#load "../common/matrixFormatting.fsx"

let elvesToDisplayable elves = 
    elves.Coordinates
    |> Seq.toList
    |> Array2D.fromPoints System.Drawing.Color.Blue System.Drawing.Color.Empty
    |> toDisplayable id

In [136]:
#!share sampleRaw --from value

parse sampleRaw |> elvesToDisplayable

The actual input:

In [137]:
#!value --name actualRaw --from-file ./data_actual.txt

In [138]:
#!share actualRaw --from value

parse actualRaw |> elvesToDisplayable |> withSettings { Width = 300 }

### Part 1

In [139]:
let a8 = [-1,-1; -1,0; -1,1; 0,-1; 0,1; 1,-1; 1,0; 1,1] |> List.map Point

type AjdCheck = { Direction: Point; CheckedPoints: Point list }

let n = { 
    Direction = Point(-1,0)
    CheckedPoints = a8 |> List.filter (Point.x >> ((=) -1))
}
let s = { 
    Direction = Point(1,0)
    CheckedPoints = a8 |> List.filter (Point.x >> ((=) 1))
}
let w = { 
    Direction = Point(0,-1)
    CheckedPoints = a8 |> List.filter (Point.y >> ((=) -1))
}
let e = { 
    Direction = Point(0,1)
    CheckedPoints = a8 |> List.filter (Point.y >> ((=) 1))
}


In [140]:
type State = { Elves : Elves; AdjChecks : AjdCheck list }

In [141]:
let proposeMove state point = 

    if a8 |> Seq.map ((+)point) |> Seq.exists state.Elves.Coordinates.Contains
    then 
        let passedCheck = 
            state.AdjChecks
            |> List.filter (
                fun check -> 
                    check.CheckedPoints 
                    |> List.map ((+)point)
                    |> List.exists state.Elves.Coordinates.Contains 
                    |> not
            
            )
            |> List.tryHead
        match passedCheck with
        | Some { Direction = direction } -> 
            point + direction
        | None -> point
    else point


let round state = 
    let newPoints = 
        state.Elves.Coordinates   
        |> Seq.group (proposeMove state) Seq.toArray
        |> Seq.collect (fun (to',from) -> if (Array.length from = 1) then [|to'|] else from)
        |> HashSet


    let currentCheck::restChecks = state.AdjChecks
    { Elves = { Coordinates = newPoints }; AdjChecks = restChecks @ [currentCheck] }

In [142]:
let infiniteRounds initState = 
    Seq.unfold (fun state -> 
        let newState = round state
        Some(newState, newState)
    ) initState

In [143]:
let initStateSample = { Elves = parse sampleRaw; AdjChecks = [n;s;w;e] }
let state10Sample = 
    infiniteRounds initStateSample |> Seq.skip 9 |> Seq.head

state10Sample.Elves
|> elvesToDisplayable
|> display

type private Tile = | Elf | Empty
let countEmpty elves =
    elves.Coordinates
    |> Seq.toList
    |> Array2D.fromPoints Elf Empty
    |> Array2D.toSeq
    |> Seq.filter ((=)Empty)
    |> Seq.length

state10Sample.Elves |> countEmpty

110

In [144]:
let initStateActual = { Elves = parse actualRaw; AdjChecks = [n;s;w;e] }
let state10actual = 
    infiniteRounds initStateActual |> Seq.skip 9 |> Seq.head

state10actual.Elves
|> elvesToDisplayable
|> withSettings { Width = 600 }
|> display

state10actual.Elves |> countEmpty

3849

### Part 2

In [150]:
let noMoveRound initState = 
    let (round, state) = 
        infiniteRounds initState
        |> Seq.indexed
        |> Seq.pairwise
        |> Seq.find (fun ((i1, prev), (i2, current)) -> prev.Elves.Coordinates.SetEquals(current.Elves.Coordinates))
        |> snd
    (round + 1), state.Elves

In [151]:
noMoveRound initStateSample |> fst |> display
noMoveRound initStateSample |> snd |> elvesToDisplayable

20

In [155]:
#!time 
let noMoveRoundActual = noMoveRound initStateActual
noMoveRoundActual |> fst

995

Wall time: 52529.2448ms

In [154]:
noMoveRoundActual |> snd 
|> elvesToDisplayable |> withSettings { Width = 600 }